In [ ]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Data integration

### URLs
- https://github.com/jorisvandenbossche/2015-PyDataParis

### Working with a Pandas dataframe

<img src="https://github.com/FIIT-IAU/2015-PyDataParis/raw/b900fdb9f3c12e9206bb417022dd004abf023c0f/img/dataframe.png" width="50%" height="50%" />


# Case study: Air quality in Europe
**[European air quality information reported by EEA member countries](https://www.eea.europa.eu/data-and-maps/data#c0=5&c11=&c5=all&b_start=0).**

AirBase (The European Air quality dataBase): hourly measurements of all air quality monitoring stations from Europe.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

In [ ]:
filename = "data/input/BETR8010000800100hour.1-1-1990.31-12-2012"
df = pd.read_csv(filename)
df.head()

Vidíme, že pri načítaní nastalo viacero problémov. Skúsme sa teda pozrieť na dáta v nejakom editore predtým, ako ich načítame:

In [ ]:
%%bash
head data/input/BETR8010000800100hour.1-1-1990.31-12-2012

z tohoto zatiaľ vieme asi len to, že pôjde o **csv formát, separátor hodnôt je \t**, sú tam samé numerické dáta a nemáme pomenované atribúty.

In [ ]:
%%bash
ls -lh data/input/BETR8010000800100hour.1-1-1990.31-12-2012

In [ ]:
%%bash 
wc -l data/input/BETR8010000800100hour.1-1-1990.31-12-2012

Takže tých dat nie je zas tak veľa a nemusím sa báť to všetko načítať do pamäte

In [ ]:
data = pd.read_csv(filename, 
                   sep='\t', 
                   header=None)
data.head()

Máme 49 stĺpcov. Dátum a 48 ďalších numerických atribútov. Každý druhý sa zdá byt binárny. Asi nejaký príznak.

Dáta sú tvorené meraniami nejakej veličiny asi v jednotlivých hodinách dňa. 

Čo deň, to riadok. Každá hodina má zvlášť stĺpec + je tu stĺpec pre nejaký príznak, ktorý nás teraz nezaujíma.

Sú tam nejaké divné hodnoty, ktoré by tam asi nemali byť: -999 a -9999.

Dátum bude asi index

In [ ]:
data = pd.read_csv(filename, 
                   sep='\t', 
                   header=None,
                   na_values=[-999, -9999], 
                   index_col=0
                  )
data.head()

In [ ]:
# skusime povyhadzovat tie flagy, ktore nas nezaujimaju. Zhodou okolnosti je to kazdy druhy stlpec
data.columns[1::2]

In [ ]:
data = data.drop(data.columns[1::2], 
                 axis=1)
data.head()

In [ ]:
# Skusme si nejak normalne pomenovat vzniknute stlpce
["{:02d}".format(i) for i in range(len(data.columns))]

In [ ]:
# mam nejako rozsypane nazvy stlpcov
data.columns = ["{:02d}".format(i) for i in range(len(data.columns))]
data.head()

**Skúsme presunúť každé meranie na samostatný riadok**

In [ ]:
data = data.stack()
data.head()

In [ ]:
# vysledok preusporiadania je viacdimenzionalny Series objekt, a nie DataFrame.
type(data)  

In [ ]:
# mohli by sme nejak normalne pomenovat stlpec, napr. nazvom meracej stanice, ktora je v nazve suboru
_, fname = os.path.split(filename)
station = fname[:7]
print(filename)
print(station)

In [ ]:
#reset index mi z toho spravi data frame
data = data.reset_index(name=station) 
# data = data.reset_index() 

print(type(data))
data.head()

In [ ]:
data = data.rename(columns = {0:'date', 'level_1':'hour'})
data.head()

In [ ]:
# teraz tomu vyrobime novy index z datumu a hodiny
data.index = pd.to_datetime(data['date'] + ' ' + data['hour'])
data.head()

In [ ]:
# zmazeme nepotrebne stlpce
data = data.drop(['date', 'hour'], axis=1)
data.head()

# Above-code for one station is inserted into the python file  `airbase.py`
**We are going to work with more stations.**

In [ ]:
import airbase
no2 = airbase.load_data()

In [ ]:
no2.head(3)

In [ ]:
no2.tail()

In [ ]:
no2.info()

In [ ]:
no2.describe()

In [ ]:
no2.plot(kind='box')

In [ ]:
# boxplot vie ukazat aj outlierov
# sns.boxplot(no2, sym='k.')
sns.boxplot(data=no2, sym='k.')

In [ ]:
no2['BETN029'].plot(kind='hist', bins=50)

In [ ]:
sns.violinplot(data=no2)

In [ ]:
# first plotting
no2.plot(figsize=(12,6))

In [ ]:
# mozem si povedat, ze chcem len nejaku mensiu cast
no2[-500:].plot(figsize=(12,6))

**Alebo použijem zaujímavejšie operácie s timeseries**

In [ ]:
# kedze index su casy, tak viem robit s nimi zaujimave veci
no2.index 

In [ ]:
# napriklad definovat rozsahy pomocou retazca s datumom
no2["2010-01-01 09:00": "2010-01-01 12:00"] 

In [ ]:
# alebo takto vybrat vsetky data z jedneho konkretneho roku
no2.loc['2012']
# no2['2012'] 
# no2['2012'].head()

# alebo len data z marca
# no2['2012/03'] 

In [ ]:
# komponenty datumu su pristupne z indexu
# no2.index.hour
no2.index.year

In [ ]:
# a co je zaujimavejsie, viem zmenit vzorkovaciu frekvenciu
no2.resample('D').mean().head()

In [ ]:
# je tu asi nejaka sezonnost?
no2.resample('M').mean().plot()

In [ ]:
# dlhodoby trend?
no2.resample('A').mean().plot()

In [ ]:
# tyzdenna sezonnost?
no2['2012-3':'2012-4'].resample('D').mean().plot()

In [ ]:
# mozem pouzit aj viacero agregacnych funkcii a porovnat si ich
no2.loc['2009':, 'FR04037'].resample('M').agg(['mean', 'median']).plot()
# no2.loc['2009':, 'FR04037'].resample('M').agg(['mean', 'std']).plot()

## Pozor resample != groupby

In [ ]:
# Toto je časový priebeh s mesačnou granularitou. Spriemerované sú hodnoty v priebehu mesiaca
no2.resample('M').mean().plot()

In [ ]:
# Toto sú spriemerované všetky hodnoty pre mesiac s daným číslom. Aj naprieč rokmi. 
# Získal som teda priemerný priebeh hodnoty počas roka s mesačnou granularitou.
no2.groupby(no2.index.month).mean().plot()

# Sumár, čo si zobrať z tejto EDA

* Uistite sa, že dáta sú kódované správne (najčastejšie sa treba pozrieť manuálne do dát)
* Uistite sa, že dáta spadajú do očakávaného rozsahu a všetky majú očakávaný tvar (napríklad formát času)
* Nikdy nemeňte dáta manuálne. Vždy používajte kód, ktorý si odložíte a použijete vždy, keď budete opakovať experiment. Chceme, aby bola analýza reprodukovateľná
* Spravte si grafy všetkého, čo sa len dá, aby ste si vizuálne potvrdili, že niečo je tak, ako by malo byt